![Banner](img/banner.png)

# **Taller N°4:** Evaluación de Clasificadores Binarios

***Matemáticas para Machine Learning***.

**Semana 4 - Práctica Calificada -** Probabilidad II

**Profesor:** *Fernando Lozano* - **Autor Notebook:** *César Garrido Urbano*

# Introducción

## Descripción

El presente *jupyter notebook* contiene todo el material para el desarrollo del Taller de la Semana 4 del curso ***Matemáticas para Machine Learning***. En este taller usted pondrá en práctica algunos de los conceptos que ha aprendido a lo largo de los módulos de probabilidad para juzgar objetivamente la calidad (o exactitud más precisamente) de modelos de Machine Learning a partir de los datos disponibles. 

**Objetivos de Aprendizaje:**


*   Repasar conceptos de probabilidad y estadística útiles para la evaluación de modelos de machine learning: Cálculo de media, y matriz de covarianza empíricas a partir de observaciones.
*   Cuantificar la cantidad de observaciones que se tienen para evaluar la calidad de los estimativos.
*   Evaluar modelos de Machine Learning considerando el número de observaciones y la precisión de los estimativos.



## Teoría


Una de las métricas más comunes y ampliamente usada al momento de evaluar un modelo de clasificación en *Machine Learning* se conoce como ***Accuracy*** (o exactitud en español, pero este es un término que no suele tener una traducción tan sencilla).

Esta métrica calcula la proporción de aciertos que tiene el modelo de clasificación con respecto al número de desaciertos. Es decir:

\begin{equation}
Accuracy = P(h(x) = y)
\end{equation}

El accuracy será la probabilidad de que el modelo $h$ prediga de forma correcta la etiqueta $y$ dados los datos $x$. En otras palabras será el cálculo de las veces que el clasificador acierta sobre el total de predicciones. Entiéndase también como el complemento del error (número de desaciertos sobre el total).

\begin{equation}
Accuracy = \frac{Aciertos}{Total}
\end{equation}


De esta manera un clasificador que con accuracy del 90% (o 0,90) ha acertado el 90% de las predicciones sobre un set de pruebas.

No obstante, esta puede llegar a ser una métrica engañosa, sobretodo cuando se tiene un set de datos desbalanceado. Esto hace referencia a cuando usted tiene muchos más datos de una clase que de otra. Por ejemplo:

Si usted tiene un total de 100 datos de prueba, de los cuales 90 datos pertenecen a la clase 1 y 10 pertenecen a la clase 0, un clasificador puede predecir siempre la clase 1 sin siquiera ver los datos y tendrá un accuracy del 90% (porque va a acertar el 90% de las veces). 

Así las cosas, usted deberá considerar también que tan balanceados están los datos al momento de realizar una evaluación de los clasificadores. 

No obstante, para estos casos existen otras métricas como la Precisión, Sensitividad o el F1-Score que verá en más detalle en sus otros cursos de *Machine Learning*.

Por otro lado, y más allá del balance de los datos, el número de datos con los que pruebe su modelo será de gran importancia. Para esto podemos utilizar las desigualdades probabilísticas estudiadas en la última lección de esta semana.

Recordemos que el cálculo del error, así como el de parámetros y del *accuracy*, resulta en estimaciones que también tienen un comportamiento probabilístico (como el de una variable aleatoria). Estas estimaciones, también sujetas a una variabilidad, se cuantifican, por ejemplo, con la desigualdad de Chernoff:

\begin{equation}
P\left[|p-\hat{p}|\geq\epsilon\right] \leq 2e^{-2\epsilon^2n}
\end{equation}

Esta desigualdad probabilística nos permite entender la relación entre el número de datos, la variabilidad del estimativo del error (o del *accuracy*) y la confianza con la que podemos realizar dichas estimación. 
Escrito de otra manera esto sería:

\begin{equation}
\delta = 2e^{-2\epsilon^2n}
\end{equation}

Donde, la confianza corresponde a $1 - \delta$.

## Metodología

Para desarrollar el taller usted deberá editar las celdas de código dispuestas para esto. Estas estarán marcadas con el siguiente comentario:

```python
# =====================================================
# COMPLETAR ===========================================
# 

# =====================================================
```

Edite o complete el códgio dentro de estas líneas de comentarios. Dentro de estos comentarios encontrará indicaciónes de lo que debe hacer, así como algunas de las variables que debe utilizar o calcular (puede que estas tengan ya una estructura para llenar o no, revise y complete la asignación).

Adicionalmente, se encontrará con preguntas a las que deberá responder a partir del trabajo que realice durante el Taller. Utilice las celdas de Markdown dispuestas para esto, estas estarán marcadas de la siguiente manera:

**Respuesta:**

# Problema

Como estudiante destacado del curso de *Matematicas para Machine Learning* usted ha sido seleccionado para apoyar al profesor en el curso de *Aprendizaje Supervisado*. Para esto, el profesor le ha pedido que lo ayude a evaluar la calidad de los modelos que entregaron algunos estudiantes como parte de la primera tarea.

Adicional a los modelos, el profesor le entrega a usted unos datos de prueba (datos que los estudiantes no han utilizado para entrenar los modelos) y es con estos datos que usted va a evaluar la calidad de los clasificadores binarios (únicamente dos clases).


## Inicialización

In [2]:
# Librerias Principales
import sys
import math as m
import numpy as np
import pandas as pd
import pickle
import sklearn
import os

## Análisis de Datos de Prueba


Primero usted analiza los datos de prueba, estos corresponden al dataset HTRU2 (dataset con el que usted trabajo parcialmente durante el taller de la Semana 3). 

Recuerde que este es un dataset que contiene datos de mediciones astronómicas de posibles pulsares. Estos son un tipo de estrella de neutrones que emite radiación detectable en la Tierra. Estas estrellas son de gran interés en distintas áreas de la física y la astronomía, pero requieren de un análisis detallado de los datos para ser detectadas. Por esto último se han empleado últimamente técnicas de Machine Learning para desarrollar modelos que las detecten. 

En este caso usted va a trabajar con todos los datos de entrada que dispone el dataset (8 variables en total) y una etiqueta de salida o clasificación (Pulsar, 1 si los datos corresponden a un pulsar y 0 de lo contrario).

In [3]:
# Cargar datos
datos_prueba = pd.read_csv("datos_prueba.csv", delimiter=";")
datos_prueba.head()

,Mean of the integrated profile,Standard Deviation of the integrated profile,Excess kurtosis of the integrated profile,Skewness of the integrated profile,Mean of the DM-SNR curve,Standard Deviation of the DM-SNR curve,Excess kurtosis of the DM-SNR curve,Skewness of the DM-SNR curve,Pulsar
0,-1.640564,-0.157200,1.394007,0.768875,1.316362,1.428186,-1.193214,-0.778742,1
1,-0.721008,2.984612,-0.060149,-0.516442,2.985452,2.232196,-1.484369,-0.793014,1
2,0.841523,-0.573540,-0.605101,-0.406732,-0.515333,-0.646135,0.224096,-0.086013,0
3,-0.810171,-1.306889,0.631106,0.470591,-0.465441,-0.268295,-0.112322,-0.393141,1
4,0.321906,0.464190,-0.599659,-0.509291,-0.564987,-1.031511,2.077373,2.854560,0


In [4]:
print(f"Número de datos de Prueba: {len(datos_prueba)}")

Número de datos de Prueba: 466


## Evaluación de Modelos

Primero usted va a calcular la exactitud (*accuracy* por su nombre en inglés) de todos los modelos. Complete el código faltante para esto e implemente las funciones requeridas.

In [5]:
# Ejemplo de cargar 1 modelo
loaded_model = pickle.load(open("modelos/modelo_1.sav", 'rb'))

In [6]:
# Cargar los modelos
modelos = []
# COMPLETAR =============================================
# =======================================================
#
# AYUDA: Carge todos los modelos en el arreglo declarado.
list_models = os.listdir('modelos/')
for i in range(0,len(list_models)):
    modelos.append(pickle.load(open("modelos/modelo_"+str(i)+".sav", 'rb')))
# =======================================================

In [7]:
modelos

[DummyClassifier(),
 GaussianNB(),
 GaussianNB(),
 DecisionTreeClassifier(),
 DecisionTreeClassifier(),
 KNeighborsClassifier(),
 KNeighborsClassifier(),
 LogisticRegression(),
 LogisticRegression()]

In [8]:
# Función para evaluar la exactitud

def calcular_accuracy(predicciones, etiquetas):
    """
    Calcula el accuracy de un arreglo de predicciones y etiquetas.
    ___________________________________
    Entrada:
    predicciones   [numpy.array] Arreglo con las etiquetas predichas.
    etiquetas      [numpy.array] Arreglo con las etiquetas correctas.
    ___________________________________
    Salida:
    accuracy       [float] Accuracy calculado.
    """
    # COMPLETAR =============================================
    # =======================================================
    #
    # AYUDA: Calcule el accuracy como el número de predicciones correctas sobre el total.
    numero_predicciones_correctas = (predicciones == etiquetas).sum()
    #print(numero_predicciones_correctas)
    # Calcular el accuracy
    accuracy = numero_predicciones_correctas / len(predicciones)
    # =======================================================

    return accuracy

In [9]:
# Calculo de accuracy para todos los modelos
dt = pd.DataFrame()
for modelo in modelos:
    # Datos y etiquetas
    X = datos_prueba.drop(columns=["Pulsar"])
    y_true = datos_prueba["Pulsar"]

    # Predicción del modelo
    y_pred = modelo.predict(X)
    # Calculo de metricas
    acc = calcular_accuracy(y_true, y_pred)
    dt = pd.concat([dt,pd.DataFrame([[modelo,acc]], columns = ['Modelo','Accuracy'])]).reset_index(drop = True)

In [10]:
dt.sort_values(by = 'Accuracy', ascending = False)

,Modelo,Accuracy
3,DecisionTreeClassifier(),0.937768
4,DecisionTreeClassifier(),0.929185
7,LogisticRegression(),0.924893
8,LogisticRegression(),0.920601
1,GaussianNB(),0.896996
6,KNeighborsClassifier(),0.862661
5,KNeighborsClassifier(),0.847639
2,GaussianNB(),0.828326
0,DummyClassifier(),0.699571


### ¿Datos desbalanceados?

A simple vista pareciera que los datos no están balanceados. Revise que tan balanceados estan esos datos de prueba. Complete la función para esto.

In [11]:
datos_prueba.Pulsar.value_counts()

0    326
1    140
Name: Pulsar, dtype: int64

In [12]:
# Evaluar balance de los datos

def calcular_proporcion(datos, nombre_etiqueta="Pulsar"):
    """
    Calcula la proporción de datos de una clase sobre el total.
    ___________________________________
    Entrada:
    datos              [pandas.DataFrame] DataFrame con los datos.
    nombre_etiqueta    [str] Nombre de la columna con las etiquetas.
    ___________________________________
    Salida:
    proporcion         [float] Porcentaje de datos de clase 1 sobre el total.
    """

    # COMPLETAR =============================================
    # =======================================================
    #
    # AYUDA: Calcule la proporción de datos de clase 1 sobre el total de datos.
    datos_clase_1 = datos[datos[nombre_etiqueta] == 1]
    numero_datos_clase_1 = len(datos_clase_1)

    # Calcular la proporción
    proporcion = numero_datos_clase_1 / len(datos)
    # =======================================================

    return proporcion


In [13]:
prop = round(calcular_proporcion(datos_prueba), 3)
print(f"Hay una proporción de {prop} datos de pulsar por cada dato de no pulsar")

Hay una proporción de 0.3 datos de pulsar por cada dato de no pulsar


### Accuracy

¿Con base en los resultados generales de los clasificadores, considera que todos son buenos? ¿Cuáles sí o cuáles no?

Justifique su respuesta considerando el balance de los datos de prueba y el resultado del accuracy sobre el set de prueba inicial.

**Respuesta:**
> Los resultados de los clasificadores muestran diferencias en el rendimiento en términos de precisión (accuracy). No se puede afirmar que todos los clasificadores sean igualmente buenos, ya que existen diferencias significativas en su capacidad para realizar clasificaciones precisas. <br>
Los clasificadores con las puntuaciones más altas de precisión son los DecisionTreeClassifier (con precisiones del 93.77% y 92.92%).Por último, DummyClassifier tiene la precisión más baja del 69.96%, pero esto no es sorprendente, ya que se trata de un clasificador "tonto" que realiza predicciones aleatorias o basadas en la clase predominante y no está diseñado para ser preciso.

> El desbalance en el conjunto de datos de prueba puede influir en la métrica de precisión (accuracy) y en la evaluación de los modelos; asi como la precisión sesgada hacia la clase mayoritaria, ya que el conjunto de datos de prueba refleja el desbalance de clases, es probable que los modelos tengan una alta precisión al predecir la clase mayoritaria (en este caso, la clase con etiqueta 0) porque hay más ejemplos de esa clase en el conjunto de datos. <br> Esto puede llevar a una métrica de precisión (accuracy) que parece alta, en sí misma puede ser engañosa en conjuntos de datos desbalanceados ya que puede dar la impresión de que un modelo es altamente preciso, pero en realidad, solo puede estar haciendo un buen trabajo al predecir la clase mayoritaria.

### ¿Suficientes datos?

Usted nota que el número de datos de prueba con los que se va a evaluar a los estudiantes parece ser reducido. Es por esto que implementa una función con la desigualdad de Chernoff para evaluar el número de datos que se necesita para un error del estimativo y una confianza dados.

In [14]:
# Función desigualdad de Chernoff

def n_datos_chernoff(epsilon, confianza):
    """
    Calcula el número de datos necesarios para cumplir con la desigualdad de Chernoff.
    ___________________________________
    Entrada:
    epsilon        [float] La variabilidad del estimativo del error/accuracy.
    confianza      [float] Confianza con la que el estimativo del error estará dentro de
                           la variabilidad estipulada.
    ___________________________________
    Salida:
    n_datos        [plt.fig] Número de datos mínimos para cumplir con la desigualdad de
                             Chernoff.
    """

    # COMPLETAR =============================================
    # =======================================================
    #
    # AYUDA: Despeje n de la relación presentada en la teoría.
    # Recuerde delta = 1-confianza
    n_datos = np.log(2/(1-confianza))/(2*epsilon*epsilon)
    # ==============================================================
    # Redondear al entero superior, i.e. función techo,
    n_datos = np.ceil(n_datos)


    # =======================================================

    return n_datos


In [15]:
# Evaluar número de datos para distintos niveles de error y confianza

c = 0.99
e = 0.05

n = n_datos_chernoff(epsilon=e , confianza=c)

print(f"Se necesitarían alrededor de {n:.0f} datos para poder asegurar un error estimado menor"
      f" a {e:.2f} con una confianza del {c:.2f}.")

Se necesitarían alrededor de 1060 datos para poder asegurar un error estimado menor a 0.05 con una confianza del 0.99.


In [16]:
df = pd.DataFrame()
for c in [0.85, 0.90, 0.95, 0.98]:
    for e in [0.05, 0.10, 0.02]:
        n = n_datos_chernoff(epsilon=e , confianza=c)
        df = pd.concat([df,pd.DataFrame([[n,e, c]], columns = ['Can_Datos_necesarios','Error_estimado', 'Confianza'])]).reset_index(drop = True)
df.sort_values('Can_Datos_necesarios', ascending = False, inplace = True)
df

,Can_Datos_necesarios,Error_estimado,Confianza
11,5757.0,0.02,0.98
8,4612.0,0.02,0.95
5,3745.0,0.02,0.90
2,3238.0,0.02,0.85
9,922.0,0.05,0.98
6,738.0,0.05,0.95
3,600.0,0.05,0.90
0,519.0,0.05,0.85
10,231.0,0.10,0.98
7,185.0,0.10,0.95


### Mejor Clasificador

¿Cuál de los modelos consideraría el mejor? ¿Por qué?

Tenga en cuenta el número de datos de prueba que tiene y la confianza con la que puede decir eso y que tanto puede afirmar eso. Varie la confianza y el error del estimativo en la función implementada.

**Respuesta:**
>  El DecisionTreeClassifier() es el mejor clasificador en este caso por las siguientes razones: <br>
- El accuracy es una métrica de evaluación importante que mide la proporción de predicciones correctas. El DecisionTreeClassifier() obtiene un accuracy de 93.7% y 92.9%, que es superior al de los otros clasificadores.
- Es un clasificador interpretable debido a que es posible entender cómo el modelo toma sus decisiones. Esto puede ser útil para comprender los resultados del modelo y para mejorar su rendimiento. <br>
Sin embargo, es importante tener en cuenta que el DecisionTreeClassifier() puede ser sensible a los datos ruidosos. Si los datos de entrenamiento contienen valores atípicos o ruido, el modelo puede verse afectado negativamente. En general, el DecisionTreeClassifier() es una buena opción para este problema. Sin embargo, es importante considerar otros factores, como la sensibilidad a los datos ruidosos, al elegir un clasificador.,

> Se realiza una tabla con una lista de posibles variaciones de epsilon y confianza para la comparación entre valores

| Can_Datos_necesarios | Error_estimado | Confianza |
|---------------------:|---------------:|----------:|
|               5757.0 |           0.02 |      0.98 |
|               4612.0 |           0.02 |      0.95 |
|               3745.0 |           0.02 |      0.90 |
|               3238.0 |           0.02 |      0.85 |
|                922.0 |           0.05 |      0.98 |
|                738.0 |           0.05 |      0.95 |
|                600.0 |           0.05 |      0.90 |
|                519.0 |           0.05 |      0.85 |
|                231.0 |           0.10 |      0.98 |
|                185.0 |           0.10 |      0.95 |
|                150.0 |           0.10 |      0.90 |
|                130.0 |           0.10 |      0.85 |

> Se evidencia que entre mas bajo es el error estimado mas cantidad de datos se necesitan y por lo que si utilizamos un error de $0.02$ y una confianza de $0.98$ se requieren un total de $5757.0$ Datos para poder respaldar esos valores, asi como si usamos un error mas grande la cantidad de datos sera menos, por lo que se evidencia una relación indirecta entre estos dos valores.

## Datos Completos

Despúes de analizar los resultados, usted habla con el profesor y este le da acceso a la base de datos completa, de donde logra descargar nuevos datos. Repita el proceso anterior y responda nuevamente a las preguntas.

In [17]:
# Cargar datos
datos_completos = pd.read_csv("datos_completos.csv", delimiter=";")
datos_completos.head()

,Mean of the integrated profile,Standard Deviation of the integrated profile,Excess kurtosis of the integrated profile,Skewness of the integrated profile,Mean of the DM-SNR curve,Standard Deviation of the DM-SNR curve,Excess kurtosis of the DM-SNR curve,Skewness of the DM-SNR curve,Pulsar
0,122.812500,39.163270,0.176058,0.782360,3.902174,21.111496,6.539226,47.987757,0
1,127.960938,54.147138,-0.018438,-0.272149,3.459866,16.956295,7.415251,72.483546,0
2,138.773438,43.032912,-0.194322,0.495927,1.165552,13.858495,14.069355,215.780933,0
3,116.539062,52.487633,0.368429,-0.113184,6.264214,30.608059,5.133793,26.603239,0
4,110.265625,45.266655,-0.066089,0.373828,3.263378,19.080187,7.432290,65.070491,0


In [25]:
datos_completos.Pulsar.value_counts()

0    8136
1     812
Name: Pulsar, dtype: int64

In [18]:
prop = round(calcular_proporcion(datos_completos), 3)
print(f"Hay una proporción de {prop} datos de pulsar por cada dato de no pulsar")

Hay una proporción de 0.091 datos de pulsar por cada dato de no pulsar


In [19]:
# Evaluar número de datos para distintos niveles de error y confianza

c = 0.95
e = 0.05
p = n_datos_chernoff(epsilon=e , confianza=c)

print(f"Se necesitarían alrededor de {p:.0f} datos para poder asegurar un error estimado menor"
      f" a {e:.2f} con una confianza del {c:.2f}.")

Se necesitarían alrededor de 738 datos para poder asegurar un error estimado menor a 0.05 con una confianza del 0.95.


In [26]:
# Calculo de accuracy para todos los modelos

accuracy = []
dt = pd.DataFrame()
for modelo in modelos:
    
    # Datos y etiquetas
    X = datos_completos.drop(columns=["Pulsar"])
    y_true = datos_completos["Pulsar"]

    # Predicción del modelo
    y_pred = modelo.predict(X)

    # Calculo de metricas
    acc = calcular_accuracy(y_true, y_pred)

    accuracy.append(acc)
    dt = pd.concat([dt,pd.DataFrame([[modelo,acc]], columns = ['Modelo','Accuracy'])]).reset_index(drop = True)
dt.sort_values('Accuracy', ascending = False, inplace = True)

In [27]:
dt

,Modelo,Accuracy
8,LogisticRegression(),0.946021
5,KNeighborsClassifier(),0.933728
6,KNeighborsClassifier(),0.922776
7,LogisticRegression(),0.912159
0,DummyClassifier(),0.909253
3,DecisionTreeClassifier(),0.909253
1,GaussianNB(),0.890814
4,DecisionTreeClassifier(),0.129750
2,GaussianNB(),0.090635


### Accuracy general

¿Con base en los resultados generales de los clasificadores, considera que todos son buenos? ¿Cuáles sí o cuáles no?

Justifique su respuesta considerando el balance de los datos de prueba.

**Respuesta:**
> Los clasificadores con las puntuaciones más altas de precisión son los LogisticRegression() con 94.6% y KNeighborsClassifier() con presicion (93.3% y 92.2%).Por último, GaussianNB() tiene la precisión más baja con 9%

> El comportamiento inusual de obtener mejores resultados de precisión en un conjunto de datos completos  desbalanceado (con una proporción de 0.091 de datos de clase 1 por cada dato de clase 0) en comparación con un conjunto de datos de prueba también desbalanceado (con una proporción de 0.3 de datos de clase 1 por cada dato de clase 0) podría deberse a varias razones:
- El tamaño del conjunto de prueba puede influir en las métricas de evaluación.
- La variabilidad en los datos también juega un papel importante, puede ser que el conjunto de prueba con la proporción de 0.091 contenga ejemplos más representativos de la población general y, por lo tanto, permita una mejor generalización del modelo.
- La división aleatoria de los datos en conjuntos de entrenamiento y prueba puede conducir a diferentes resultados en función de la semilla de aleatorización. En ocasiones, una división aleatoria puede favorecer la generalización del modelo.
- La calidad de los datos y la presencia de ruido también pueden influir en los resultados, puede haber diferencias en la calidad o la limpieza de los datos entre los dos conjuntos, lo que afecta el rendimiento del modelo.

### Mejor Clasificador

¿Cuál de los modelos consideraría el mejor? ¿Por qué?

Tenga en cuenta el número de datos de prueba que tiene y la confianza con la que puede decir eso y que tanto puede afirmar eso.

**Respuesta:**
>  El LogisticRegression() con 0.94602 es el mejor clasificador en este caso por las siguientes razones: <br>
- La regresión logística es un modelo lineal que se puede entender y explicar fácilmente, los coeficientes de la regresión logística se pueden interpretar como el impacto de cada característica en la probabilidad de pertenecer a una clase.
- Los modelos de clasificación tienden a sesgarse hacia la clase mayoritaria debido a la abundancia de ejemplos de esa clase. Como resultado, pueden predecir principalmente la clase mayoritaria y tener dificultades para identificar y clasificar correctamente la clase minoritaria, es probable que los modelos tengan altas tasas de falsos negativos en la clase minoritaria. Esto significa que es más probable que clasifiquen incorrectamente ejemplos de la clase 1 como pertenecientes a la clase 0.

> Se realiza una tabla con una lista de posibles variaciones de epsilon y confianza para la comparación entre valores

| Can_Datos_necesarios | Error_estimado | Confianza |
|---------------------:|---------------:|----------:|
|               5757.0 |           0.02 |      0.98 |
|               4612.0 |           0.02 |      0.95 |
|               3745.0 |           0.02 |      0.90 |
|               3238.0 |           0.02 |      0.85 |
|                922.0 |           0.05 |      0.98 |
|                738.0 |           0.05 |      0.95 |
|                600.0 |           0.05 |      0.90 |
|                519.0 |           0.05 |      0.85 |
|                231.0 |           0.10 |      0.98 |
|                185.0 |           0.10 |      0.95 |
|                150.0 |           0.10 |      0.90 |
|                130.0 |           0.10 |      0.85 |

> Se evidencia que entre mas bajo es el error estimado mas cantidad de datos se necesitan y por lo que si utilizamos un error de $0.02$ y una confianza de $0.98$ se requieren un total de $5757.0$ Datos para poder respaldar esos valores, asi como si usamos un error mas grande la cantidad de datos sera menos, por lo que se evidencia una relación indirecta entre estos dos valores.